In [0]:
#GPUを認識しているか確認
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
#ファイルの読み書きを可能にする
from google.colab import drive
drive.mount("/content/drive")

#ファイルを置いてあるパスを入力
file_path = "/content/drive/My Drive/"

In [0]:
import keras
from keras.layers import Input, Dense, Activation, BatchNormalization, Reshape, UpSampling2D, Conv2D, MaxPool2D, Flatten
from keras.models import Model
from keras.optimizers import Adam
import numpy as np

#画像生成器
class Generator():
    def __init__(self, z_dim):
        inputs = Input(shape=(z_dim,))
        x = Dense(1024)(inputs)
        x = Activation("tanh")(x)
        x = Dense(128*7*7)(x)
        x = BatchNormalization()(x)
        x = Activation("tanh")(x)
        x = Reshape((7, 7, 128))(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(64, 5, padding="same")(x)
        x = Activation("tanh")(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(1, 5, padding="same")(x)
        predictions = Activation("tanh")(x)
        self.generator = Model(inputs, predictions)

    def get_model(self):
        return self.generator


#画像判別器
class Discriminator():
    def __init__(self, height, width, channels):
        inputs = Input(shape=(height, width, channels))
        x = Conv2D(64, 5, padding="same")(inputs)
        x = Activation("tanh")(x)
        x = MaxPool2D()(x)
        x = Conv2D(128, 5)(x)
        x = Activation("tanh")(x)
        x = MaxPool2D()(x)
        x = Flatten()(x)
        x = Dense(1024)(x)
        x = Activation("tanh")(x)
        x = Dense(1)(x)
        predictions = Activation("sigmoid")(x)
        self.discriminator = Model(inputs, predictions)

    def get_model(self):
        return self.discriminator


class DCGAN():
    def __init__(self, z_dim, height, width, channels):
        self.z_dim = z_dim
        #生成器を作成
        g = Generator(z_dim)
        self.generator = g.get_model()
        #判別器を作成
        d = Discriminator(height, width, channels)
        self.discriminator = d.get_model()
        #判別器をコンパイル
        discriminator_optimizer = Adam(lr=0.0002, beta_1=0.5)
        self.discriminator.compile(loss="binary_crossentropy", optimizer=discriminator_optimizer)
        #判別器のパラメータを固定
        self.discriminator.trainable = False
        #DCGANを作成
        inputs = Input(shape=(z_dim,))
        outputs = self.discriminator(self.generator(inputs))
        self.dcgan = Model(inputs, outputs)
        #DCGANをコンパイル
        dcgan_optimizer = Adam(lr=0.0002, beta_1=0.5)
        self.dcgan.compile(loss="binary_crossentropy", optimizer=dcgan_optimizer)

    def train(self, real_images, batch_size):
        #判別器が偽物を検出できるように訓練
        noise = np.random.uniform(-1, 1, size=(batch_size, self.z_dim))
        generated_images = self.generator.predict(noise)
        labels = np.zeros((batch_size, 1))
        d_loss_fake = self.discriminator.train_on_batch(generated_images, labels)
        #判別器が本物を検出できるように訓練
        labels = np.ones((batch_size, 1))
        d_loss_real = self.discriminator.train_on_batch(real_images, labels)
        d_loss = np.add(d_loss_fake, d_loss_real) / 2.0
        #生成器が判別器をだますことができるように訓練
        labels = np.ones((batch_size, 1))
        g_loss = self.dcgan.train_on_batch(noise, labels)
        return d_loss, g_loss

    #noiseに対する予測値を生成
    def predict(self, noise):
        return self.generator.predict(noise)

    #学習結果の読み込み
    def load_weights(self, file_path, by_name=False):
        self.dcgan.load_weights(file_path, by_name)

    #学習結果の保存
    def save_weights(self, file_path, overwrite=True):
        self.dcgan.save_weights(file_path, overwrite)

In [0]:
import os
import keras
from keras.datasets import mnist
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

#ピクセル値を-1から1に正規化
def normalize(X):
    return (X-127.5) / 127.5

#ピクセル値を0から255に戻す
def denormalize(X):
    return (X+1.0) * 127.5

def train(z_dim, height, width, channels, epochs):
    os.makedirs(file_path+"Model", exist_ok=True) #Modelフォルダの作成
    dcgan = DCGAN(z_dim, height, width, channels)
    (x_train, y_train), (_, _) = mnist.load_data() #60000枚の訓練画像データ
    x_train = x_train.reshape(x_train.shape[0], height, width, channels).astype("float32")
    x_train = normalize(x_train)
    batch_size = 128
    iterations = x_train.shape[0] // batch_size #iterations=468
    save_interval = 12 #irerationsの任意の約数
    noise = np.random.uniform(-1, 1, size=(64, z_dim)) #-1から1の一様乱数
    for epoch in range(epochs):
        print("epoch{} start".format(str(epoch)))
        for iteration in range(iterations):
            real_images = x_train[iteration*batch_size:(iteration+1)*batch_size]
            d_loss, g_loss = dcgan.train(real_images, batch_size)
            #損失値の保存
            if epoch == 0 and iteration == 0:
                with open(file_path+"Loss.txt", "a") as f:
                    f.write(str(d_loss) + "," + str(g_loss) + "\n")
            elif (iteration+1) % save_interval == 0:
                with open(file_path+"Loss.txt", "a") as f:
                    f.write(str(d_loss) + "," + str(g_loss) + "\n")
        print("epoch{} end".format(str(epoch)))
        dcgan.save_weights(file_path+"Model/dcgan_epoch{}.h5".format(str(epoch)))
        predict(z_dim, height, width, channels, epoch, noise)

def predict(z_dim, height, width, channels, epoch, noise):
    os.makedirs(file_path+"Generated", exist_ok=True) #generatedフォルダの作成
    dcgan = DCGAN(z_dim, height, width, channels)
    dcgan.load_weights(file_path+"Model/dcgan_epoch{}.h5".format(str(epoch)))
    generated_images = dcgan.predict(noise)
    images = [] #imagesの初期化
    for generated_image in generated_images:
        images.append(image.array_to_img(denormalize(generated_image), scale=False)) #ndarrayからPIL形式に変換
    save_image(images, epoch)

def save_image(images, epoch):
    H = 8 #縦に並べる個数
    W = 8 #横に並べる個数
    fig = plt.figure(figsize=(H, W))
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0, wspace=0)
    for i,image in enumerate(images):
        ax = fig.add_subplot(H, W, i+1, xticks=[], yticks=[])
        ax.imshow(image, cmap="gray")
    plt.savefig(file_path+"Generated/{}.jpg".format(epoch))
    plt.close()

def create_gif(epochs):
    images = []
    for epoch in range(epochs):
        images.append(Image.open(file_path+"Generated/{}.jpg".format(epoch)))
    images[0].save(file_path+"DCGAN.gif", save_all=True, append_images=images[1:], optimize=False, duration=200, loop=0)

def display_loss(epochs):
    #損失値の取り出し
    d_loss = []
    g_loss = []
    with open(file_path+"Loss.txt", "r") as f:
        for line in f:
            data = line.split(",")
            d_loss.append(float(data[0]))
            g_loss.append(float(data[1]))
    os.remove(file_path+"Loss.txt")
    #損失値の表示
    x = np.linspace(0, (len(d_loss)-1)*12, len(d_loss))
    plt.plot(x, d_loss)
    plt.plot(x, g_loss)
    plt.title("Discriminator loss vs Generator loss")
    plt.ylabel("loss")
    plt.xlabel("iteration")
    plt.xticks(np.linspace(0, (len(d_loss)-1)*12, int(epochs/2)+1)) #x軸のメモリの設定
    plt.grid()
    plt.legend(["D_loss", "G_loss"], loc="best")
    plt.savefig(file_path+"Loss.jpg")
    plt.show()

def main():
    z_dim = 100 #潜在変数の次元数
    height = 28 #高さ28
    width = 28 #幅28
    channels = 1 #グレースケール画像
    epochs = 20 #学習させる回数
    train(z_dim, height, width, channels, epochs)
    create_gif(epochs)
    display_loss(epochs)


if __name__ == "__main__":
    main()